# Discrete Random Walk

Consider a discrete random walk.

Let $X_t = X_{t+1} + \epsilon_t$, with $p(\epsilon_t = 1) = 0.5$ and $p(\epsilon_t = -1) = 0.5$ be the stepwise random walk. All paths are equally likely, so that the probability of winding up at a certain point can be calculated by looking at the fractions of all paths that lead to that point.

Consider the grid. The possible states that the walk can reach lie on a rotated rectangular grid. Use the coordinates:

\begin{equation}
X_t = y-x\\
t = y+x
\end{equation}

From which follows:
\begin{equation}
x = \frac{t-X_t}{2}\\
y = \frac{t+X_t}{2}
\end{equation}

The total amount of paths that lead from $(0,0)$ to $(t,X_t)$ are then given by:

\begin{equation}
n(t,X_t) = \left(\begin{array}{c}x+y\\x\end{array}\right) = \left(\begin{array}{c}x+y\\y\end{array}\right) =  \left(\begin{array}{c}t\\\frac{t+X_t}{2}\end{array}\right) = \left(\begin{array}{c}t\\\frac{t-X_t}{2}\end{array}\right)
\end{equation}

If $\frac{t\pm X_t}{2}$ are integer and 0 otherwise. 


The probability distribution of $P(X_t = s)$ could be calculated by calculating the convolution of the probability distributions for each of the $\epsilon_t$ for $t \in (1,t)$.